# 我們使用多層感知器完成 MNIST 手寫體數字圖片資料集 [LeCun1998] 的分類任務。
<img src="https://tf.wiki/_images/mnist_0-9.png">

# 資料獲取及預處理： tf.keras.datasets
## 先進行預備工作，實現一個簡單的 MNISTLoader 類來讀取 MNIST 資料集資料。這裡使用了 tf.keras.datasets 快速載入 MNIST 資料集。

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten層將除第一維（batch_size）以外的維度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)   # 正規化output
        return output

<img src="https://tf.wiki/_images/mlp.png" height="50%">
MLP模型示意圖

## 模型的訓練： tf.keras.losses 和 tf.keras.optimizer
### 定義一些模型超參數 (Hyperparameters)：

In [11]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [12]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### 然後疊代進行以下步驟：

1.從 DataLoader 中隨機取一批訓練資料；

2.將這批資料送入模型，計算出模型的預測值；

3.將模型預測值與真實值進行比較，計算損失函數（loss）。這里使用 tf.keras.losses 中的交叉熵函數作為損失函數；

4.計算損失函數關於模型變數的導數；

5.將求出的導數值傳入優化器，使用優化器的 apply_gradients 方法更新模型參數以最小化損失函數

In [ ]:
num_batches = int(data_loader.num_train_data // batch_size*num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

In [14]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.972100


# 卷積神經網路（CNN）
## 卷積神經網路 （Convolutional Neural Network, CNN）是一種結構類似於人類或動物的 視覺系統 的神經網路，包含一個或多個卷積層（Convolutional Layer）、池化層（Pooling Layer）和全連接層（Fully-connected Layer）。

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5,5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7*7*64,))
        self.dense1 = tf.keras.layers.Dense(units=1021, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self,inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32] 因為padding,所以一樣是 28*28
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [5]:
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
num_batches = int(data_loader.num_train_data // batch_size*num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

In [7]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.992300


<img src="https://tf.wiki/_images/cnn.png">

## 可以發現準確率相較於前節的多層感知器有非常顯著的提高。事實上，通過改變模型的網路結構（比如加入 Dropout 層防止過擬合），準確率還有進一步提升的空間。

# 使用 Keras 中預定義的典型卷積神經網路結構

## tf.keras.applications 中有一些預定義好的典型卷積神經網路結構，如 VGG16 、 VGG19 、 ResNet 、 MobileNet 等。我們可以直接呼叫這些典型的卷積神經網路結構（甚至載入預訓練的參數），而無需手動定義網路結構。